**Predictive Lead Scoring**

# import useful libraries and Load the data

In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [2]:
sys.path.append("..")
data_dir = Path('../data/') 
modules_dir = Path('../modules/')

> The dataset: https://www.kaggle.com/arashnic/banking-loan-prediction

**The content of the dataset**
- ID : Unique Customer ID
- Gender : Gender of the applicant
- DOB : Date of Birth of the applicant
- LeadCreationDate : Date on which Lead was created
- City_Code : Anonymised Code for the City
- City_Category: Anonymised City Feature
- Employer_Code: Anonymised Code for the Employer
- Employer_Category1 : Anonymised Employer Feature
- Employer_Category2: Anonymised Employer Feature
- Monthly_Income : Monthly Income in Dollars
- CustomerExistingPrimaryBankCode : Anonymised Customer Bank Code
- PrimaryBankType: Anonymised Bank Feature
- Contacted: Contact Verified (Y/N)
- Source : Categorical Variable representing source of lead
- Source_Category: Type of Source
- Existing_EMI : EMI of Existing Loans in Dollars
- Loan_Amount: Loan Amount Requested
- Loan_Period: Loan Period (Years)
- Interest_Rate: Interest Rate of Submitted Loan Amount
- EMI: EMI of Requested Loan Amount in dollars
- Var1: Anonymized Categorical variable with multiple levels
- Approved: (Target) Whether a loan is Approved or not (1-0) . Customer is Qualified Lead or not (1-0)

In [3]:
train = pd.read_csv(data_dir/'train.csv')

In [4]:
test = pd.read_csv(data_dir/'test.csv')

# Data Wrangling

## Data exploring

**samples**

In [5]:
train.sample(3)

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,...,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
64177,APPV20435377141,Female,21/03/79,24/09/16,C10005,A,COM0003290,C,4.0,2100.0,...,Y,S122,G,0.0,35000.0,4.0,NaN,NaN,2,0
9343,APPN70416965118,Male,02/02/85,14/07/16,C10003,A,COM0000028,B,3.0,3400.0,...,Y,S133,C,0.0,40000.0,3.0,NaN,NaN,7,0
35782,APPS80884347811,Female,06/03/91,21/08/16,C10010,A,COM0035676,A,4.0,3300.0,...,Y,S143,C,0.0,57000.0,4.0,NaN,NaN,7,0


In [6]:
test.sample(3)

,ID,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,...,Primary_Bank_Type,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1
21861,APPU30038923226,Male,17/03/86,02/09/16,C10007,A,COM0015820,A,4.0,1600.0,...,NaN,Y,S122,G,0.0,NaN,NaN,NaN,NaN,2
9998,APPV20184945929,Female,11/08/81,15/07/16,C10059,B,COM0022609,A,4.0,3300.0,...,G,N,S133,B,0.0,NaN,NaN,NaN,NaN,7
14084,APPD60127237945,Male,30/12/85,01/08/16,C10004,A,COM0001250,B,4.0,3450.0,...,G,Y,S133,B,0.0,10000.0,3.0,NaN,NaN,10


**target and features**

- Our target is the column **Approved** from the train table: it consists of 0 if the loan default exist and the loan is not approved and 1 if not.
- The remaining columns 21 will be considered as features and after a statistical analysis we will selected only the best ones.

**size and shape**

In [7]:
train.shape

(69713, 22)

In [8]:
test.shape

(30037, 21)

> Let us perform all our operation on train data and create a modular code for the test dataset and futures tests 

**missing values and types of columns**

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69713 entries, 0 to 69712
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   ID                                   69713 non-null  object 
 1   Gender                               69713 non-null  object 
 2   DOB                                  69698 non-null  object 
 3   Lead_Creation_Date                   69713 non-null  object 
 4   City_Code                            68899 non-null  object 
 5   City_Category                        68899 non-null  object 
 6   Employer_Code                        65695 non-null  object 
 7   Employer_Category1                   65695 non-null  object 
 8   Employer_Category2                   65415 non-null  float64
 9   Monthly_Income                       69713 non-null  float64
 10  Customer_Existing_Primary_Bank_Code  60322 non-null  object 
 11  Primary_Bank_Type           

> This helps us understand the data columns: 22
>- the DOB and Lead_creation_date are in wrong format : they should be datetime
>- Multiple columns have null values even if we are lucky to have all our target.

**number of unique values**

In [10]:
train.nunique().sort_values()

Approved                                   2
Gender                                     2
Contacted                                  2
Primary_Bank_Type                          2
City_Category                              3
Employer_Category1                         3
Employer_Category2                         4
Var1                                       5
Loan_Period                                6
Source_Category                            7
Source                                    29
Customer_Existing_Primary_Bank_Code       57
Interest_Rate                             72
Lead_Creation_Date                        92
Loan_Amount                              196
City_Code                                678
EMI                                     2179
Existing_EMI                            3245
Monthly_Income                          5010
DOB                                    10759
Employer_Code                          36617
ID                                     69713
dtype: int

> We can observe that almost all variable are Categorical because they have less unique values

## Data Cleaning

**mistype**

In [11]:
# Changing DOB and lead creation date to the appropriate type
def handle_datetime(data):
    data.DOB = data.DOB.astype('datetime64')
    data.Lead_Creation_Date = data.Lead_Creation_Date.astype('datetime64')
    return data

In [12]:
clean_train = handle_datetime(train)

> we could have parsed those columns to datetime while reading with pandas.

**missing values**

In [13]:
pd.concat([train.isnull().sum().sort_values(),train.isnull().sum().sort_values()/train.shape[0]], axis=1)\
                        .rename(columns={0:'count', 1:'percentage'})

,count,percentage
ID,0,0.000000
Source_Category,0,0.000000
Source,0,0.000000
Contacted,0,0.000000
Var1,0,0.000000
Monthly_Income,0,0.000000
Approved,0,0.000000
Lead_Creation_Date,0,0.000000
Gender,0,0.000000
DOB,15,0.000215


train.Existing_EMI

Our first strategy is to keep all the observation.
- For the missing date of birth we will first convert the date to age and then use the median to replace the null value.
- Since most of those columns are categorical, we will create a category "unknown" where the value is missing.
- For numerical columns ( Loan Amount, load period, interest rate, existing EMI and EMI ) , we will use different approach. We will assume that if there is no load amount/ period then it is 0 there and the client doesn't have any loan

In [14]:
from datetime import date

def handle_missing_val(data):
    #categorical
    
    # First let us create age columns instead of DOB so that we can replace by median
    data['age'] = data.DOB.apply(lambda x : date.today().year - x.year)
    #todo create a column for lead creation date age!!!!!
    # drop the DOB columns
    data = data.drop(columns='DOB')
    # Replacing the missing age with median
    data['age'] = data.age.fillna(data.age.median())
    
    # Replacing the other cat value with "unknown"
    cols_cat = ['City_Category', 'City_Code', 'Employer_Category1',
          'Employer_Code', 'Employer_Category2', 'Primary_Bank_Type',
          'Customer_Existing_Primary_Bank_Code']
    data[cols_cat] = data[cols_cat].fillna("unknown")
    
    # Now handle the num cols
    cols_num = ['Existing_EMI', 'Loan_Amount', 'Loan_Period', 'Interest_Rate', 'EMI']
    data[cols_num] = data[cols_num].fillna(0)
    
    return data

In [15]:
clean_train = handle_missing_val(clean_train)

In [16]:
clean_train.isnull().sum()

ID                                     0
Gender                                 0
Lead_Creation_Date                     0
City_Code                              0
City_Category                          0
Employer_Code                          0
Employer_Category1                     0
Employer_Category2                     0
Monthly_Income                         0
Customer_Existing_Primary_Bank_Code    0
Primary_Bank_Type                      0
Contacted                              0
Source                                 0
Source_Category                        0
Existing_EMI                           0
Loan_Amount                            0
Loan_Period                            0
Interest_Rate                          0
EMI                                    0
Var1                                   0
Approved                               0
age                                    0
dtype: int64

In [24]:
# save the data
# clean_train.to_csv(data_dir/'clean_train.csv', index=False)

# EDA

## Feature distributions

> We will display only for relevant columns. Id columns for example seems not relevant for example

In [23]:
feature_list = clean_train.drop(columns='Approved').columns
target = ['Approved']

cat_cols = ['Gender', 'Lead_Creation_Date', 'City_Code', 'City_Category',
       'Employer_Code', 'Employer_Category1', 'Employer_Category2',
       'Customer_Existing_Primary_Bank_Code',
       'Primary_Bank_Type', 'Contacted', 'Source', 'Source_Category',
       'Var1']
num_cols = ['Monthly_Income', 
       'Existing_EMI', 'Loan_Amount', 'Loan_Period', 'Interest_Rate', 'EMI',
        'age']

**Histograms of the categorical features:**

**Histograms of the numerical features:**

## Target distribution

## Relations : Features multicollinearity, relation between target and features

In [ ]:
fig, axs = plt.subplots(len(cat_cols), 1, figsize=(17,3), dpi= 80)

for i_feature, feature in enumerate(cat_cols):
    
    for i_loan_approval, loan_approval in enumerate([0, 1]):
        clean_train[cat_cols][clean_train[target]==loan_approval].plot(kind='density', label=loan_approval, c=['red', 'green'][i_loan_approval], ax=axs[i_feature])
    
    axs[i_feature].set_xlabel(cat_cols[i_feature])
    axs[i_feature].legend(title='Loan Approval')
    
plt.show()   